In [ ]:
%pip install pyodbc
%pip install pymssql

In [ ]:
import pyodbc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

load_dotenv()

SERVER = os.getenv("SERVER")
DATABASE = os.getenv("DATABASE")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

# สร้าง connection string สำหรับ SQLAlchemy
connection_str = f"mssql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes"

# สร้าง SQLAlchemy engine
engine = create_engine(connection_str)

SQL_QUERY = """
SELECT
    employee.marital AS marital,
    employee.band AS band,
    employee.birthdate AS birth_date,
    CONVERT(DATETIME, employee.createdAt) AS created_date,
    position.positionEng AS position_name,
    company.nameEng AS company_name,
    businessUnit.nameEng AS business_unit_name,
    division.nameEng AS division_name,
    department.nameEng AS department_name,
    section.nameEng AS section_name
FROM
    employee AS employee
    LEFT JOIN position AS position ON position.id = employee.positionId
    LEFT JOIN company AS company ON company.id = employee.companyId
    LEFT JOIN businessUnit AS businessUnit ON businessUnit.id = employee.businessUnitId
    LEFT JOIN division AS division ON division.id = employee.divisionId
    LEFT JOIN department AS department ON department.id = employee.departmentId
    LEFT JOIN section AS section ON section.id = employee.sectionId
WHERE
    position.positionEng IS NOT NULL
    AND employee.marital IS NOT NULL
"""

# ใช้ SQLAlchemy engine แทน pyodbc connection
df = pd.read_sql(SQL_QUERY, engine)

# นับจำนวนพนักงานตามสถานภาพสมรส
marital_counts = df["marital"].value_counts().sort_values(ascending=False)
# สร้างกราฟแท่งแสดงจำนวนพนักงานตามสถานภาพสมรส
plt.figure(figsize=(12, 6))
sns.barplot(
    x=marital_counts.index,
    y=marital_counts.values,
    hue=marital_counts.index,
    palette="viridis",
    legend=False,
)

# ปรับแต่งกราฟ
plt.title("Number of Employees by Marital Status", fontsize=16)
plt.xlabel("Marital Status", fontsize=12)
plt.ylabel("Number of Employees", fontsize=12)

# เพิ่มจำนวนบนแท่งกราฟ
for i, v in enumerate(marital_counts.values):
    plt.text(i, v, str(v), ha="center", va="bottom", fontsize=10)

# แสดงกราฟ
plt.tight_layout()
plt.show()